In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [3]:
all_legislative_year_url = "https://www.tbmm.gov.tr/tutanak/tutanaklar.htm"
all_legislative_year_req = requests.get(all_legislative_year_url)
all_legislative_year_bs = BeautifulSoup(all_legislative_year_req.text)
all_legislative_year_urls = all_legislative_year_bs.find_all("table")[0].find_all("a")
all_legislative_year_urls = [x["href"] for x in all_legislative_year_urls]

In [4]:
def get_year_page(legislative_year_url):
    page = requests.get(legislative_year_url)
    page_bs = BeautifulSoup(page.text, from_encoding="utf8")
    
    rows = page_bs.find_all("table")[1].find_all("tr")
    links_list = []
    for row in rows:
      if (row.find_all("td")[5].text == '\xa0'):
        print("it is empty")
      else:
        link = row.find_all("td")[5].find("a")["href"] 
        links_list.append(link)
        print(link)
    print(links_list)
    return links_list

In [5]:
# install chromium, its driver, and selenium
# !apt update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install selenium
# set options to be headless, ..
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
chromedriver_path = r"C:\Users\da12\Documents\Python Work\KamusalVeri\meclis\meclis-genel-kurul-tutanaklar-acik-oylama\chromedriver"
wd = webdriver.Chrome(chromedriver_path, options=options)
page = wd.get("http://gazetetirajlari.com/HaftalikTirajlar.aspx")

In [95]:
def get_results(links_list, url):
    votes_list = []
    summary_tables_list = []
    for link in links_list:
        print(link)
        voting_result_page = requests.get(link)
        votes_page = wd.get(link)
        # voting_result_page_bs = BeautifulSoup(voting_result_page.text)
        # draft_laws = wd.find_elements_by_tag_name("a")
        # draft_law_tables = wd.find_elements_by_xpath('//*[@id="tblTbmmOylama"]')
        meeting_name = wd.find_elements_by_tag_name("p")[2].text
        meeting_date_given = wd.find_elements_by_tag_name("p")[3].text
        draft_law_panels = wd.find_elements_by_class_name("panel")
        for draft_law_panel in draft_law_panels:
            law_name = draft_law_panel.find_elements_by_tag_name("h3")[0].get_attribute("innerText")
            print(law_name)
            summary_table = draft_law_panel.find_element_by_id("oySonuc")
            law_data_table = draft_law_panel.find_element_by_id("tblTbmmOylama")
            
            # Use the code in commented part to extract info from summary tables later. For now, you save them as raw
            # html tables
            # # summary_table_json
            # summary_table_json = {}
            # summary_table_json["law_name"] = law_name
            # summary_table_json["url"] = url
            # summary_table_json["uye_sayisi"] = summary_table.find_elements_by_tag_name("tr")[0].find_elements_by_tag_name("td")[1]
            # summary_table_json["kabul_edenler"] = summary_table.find_elements_by_tag_name("tr")[1].find_elements_by_tag_name("td")[1]
            # summary_table_json["reddedenler"] = summary_table.find_elements_by_tag_name("tr")[2].find_elements_by_tag_name("td")[1]
            # summary_table_json["cekimserler"] = summary_table.find_elements_by_tag_name("tr")[3].find_elements_by_tag_name("td")[1]
            # summary_table_json["gecersiz"] = summary_table.find_elements_by_tag_name("tr")[4].find_elements_by_tag_name("td")[1]
            # summary_table_json["mukerrer"] = summary_table.find_elements_by_tag_name("tr")[5].find_elements_by_tag_name("td")[1]
            # summary_table_json["toplam"] = summary_table.find_elements_by_tag_name("tr")[6].find_elements_by_tag_name("td")[1]
            # 
            # summary_table_json["acik_uyelikler"] = summary_table.find_elements_by_tag_name("tr")[7].find_elements_by_tag_name("td")[1]
            # summary_table_json["sira_sayisi"] = summary_table.find_elements_by_tag_name("tr")[7].find_elements_by_tag_name("td")[3]
            # summary_table_json["birlesim"] = summary_table.find_elements_by_tag_name("tr")[7].find_elements_by_tag_name("td")[4]
            # summary_table_json["birlesim_tarihi"] = summary_table.find_elements_by_tag_name("tr")[7].find_elements_by_tag_name("td")[5]
            # summary_table_json["kanunlasma_status"] = summary_table.find_elements_by_tag_name("tr")[7].find_elements_by_tag_name("td")[6]

            vote_table = law_data_table.get_attribute("outerHTML")
            vote_pd = pd.read_html(vote_table, header=1)[0]
            vote_pd["url"] = url
            vote_pd["law_name"] = law_name
            vote_pd["meeting_name"] = meeting_name
            vote_pd["meeting_date_given"] = meeting_date_given
            summary_tables_list.append(summary_table.get_attribute("outerHTML"))
            votes_list.append(vote_pd)



        # for draft_law_table in draft_law_tables:
        #     # law_name = draft_law_table.text
        #     vote_table = draft_law_table.get_attribute("outerHTML")
        #     vote_pd = pd.read_html(vote_table, header=1)[0]
        #     
        #     # vote_pd["ilgili_kanun"] = law_name
        #     # assemble_no = wd.find_elements_by_tag_name("div")[0].find_elements_by_tag_name("span")[2].text
        #     # vote_pd["assemble_no"] = assemble_no
        #     vote_pd["url"] = url
        #     # print(assemble_no, law_name)
        #     votes_list.append(vote_pd)
        #     # except ElementNotInteractableException:
        #     #     print("LINK NOT SUITABLE")
    return votes_list, summary_tables_list

In [110]:
year_votes_list = []
year_summary_tables_list = []
failed_legislative_year_urls = []
for url in all_legislative_year_urls[6:8]:
    print(url)
    # try:
    year_page = get_year_page(url)
    
    votes_obj = get_results(year_page, url)
    votes_list = votes_obj[0]
    summary_tables_list = votes_obj[1]
    print(votes_list)
    year_votes_list.append(votes_list)
    year_summary_tables_list.append(summary_tables_list)
    # except IndexError:
    #     year_page = None
    #     failed_legislative_year_urls.append(url)

https://www.tbmm.gov.tr/tutanak/donem25/tutanak2.htm
it is empty
[]
[]
https://www.tbmm.gov.tr/tutanak/donem25/tutanak1.htm
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
[]
[]


In [103]:
donem_25.describe()

,İl,Soyadı,Adı,Parti,Sistem,Pusula,Sonuç,Tarih,Saati,url,law_name,meeting_name,meeting_date_given
count,173262,173262,173262,173262,173262,173262,173262,173262,173262,173262,173262,173262,173262
unique,81,453,377,6,4,14,6,44,7913,2,310,36,41
top,İSTANBUL,AYDIN,Mehmet,Adalet ve Kalkınma Partisi,KATILMADI,-,KATILMADI,--.--.----,--:--:--,https://www.tbmm.gov.tr/tutanak/donem26/tutana...,(S.Sayısı: 359) Türkiye Cumhuriyeti Gümrük ve ...,76'ncı Birleşim,1 Mart 2017 Çarşamba
freq,27826,2219,6023,100391,111478,165916,104565,111478,111478,121853,2196,22509,22509


In [108]:
# Collected list of dfs will be in two level deep. Structure is from bottom to up:
# Vote on law -> Meeting -> Year -> year_votes_list
# Exclude empty values from list
year_urls = list(filter(None, year_votes_list))
import itertools
# Iteration for flattening list of lists
flatten = list(itertools.chain(*year_urls))
# Concatting them to a big df
donem_25 = pd.concat(flatten)

ValueError: No objects to concatenate

In [109]:
donem_25.to_excel("donem25.xlsx")

NameError: name 'donem_25' is not defined

,İl,Soyadı,Adı,Parti,Sistem,Pusula,Sonuç,Tarih,Saati,ilgili_kanun,assemble_no
0,ADANA,ÇELİK,Ömer,Adalet ve Kalkınma Partisi,KATILMADI,-,KATILMADI,--.--.----,--:--:--,(S.Sayısı: 413) Türkiye Varlık Fonu Kurulması ...,128'inci Birleşim
2,ADANA,DANIŞ BEŞTAŞ,Meral,Halkların Demokratik Partisi,KATILMADI,-,KATILMADI,--.--.----,--:--:--,(S.Sayısı: 413) Türkiye Varlık Fonu Kurulması ...,128'inci Birleşim
4,ADANA,KARAKAYA,Mevlüt,Milliyetçi Hareket Partisi,KATILMADI,-,KATILMADI,--.--.----,--:--:--,(S.Sayısı: 413) Türkiye Varlık Fonu Kurulması ...,128'inci Birleşim
5,ADANA,KÜÇÜKCAN,Talip,Adalet ve Kalkınma Partisi,KATILMADI,-,KATILMADI,--.--.----,--:--:--,(S.Sayısı: 413) Türkiye Varlık Fonu Kurulması ...,128'inci Birleşim
6,ADANA,ÖZDİŞ,İbrahim,Cumhuriyet Halk Partisi,KATILMADI,-,KATILMADI,--.--.----,--:--:--,(S.Sayısı: 413) Türkiye Varlık Fonu Kurulması ...,128'inci Birleşim
...,...,...,...,...,...,...,...,...,...,...,...
528,ŞIRNAK,ENCU,Ferhat,Halkların Demokratik Partisi,KATILMADI,RED,RED,--.--.----,--:--:--,Başbakan Ahmet Davutoğlu tarafından kurulan 64...,7'nci Birleşim
529,ŞIRNAK,İRMEZ,Aycan,Halkların Demokratik Partisi,KATILMADI,RED,RED,--.--.----,--:--:--,Başbakan Ahmet Davutoğlu tarafından kurulan 64...,7'nci Birleşim
530,ŞIRNAK,SARIYILDIZ,Faysal,Halkların Demokratik Partisi,KATILMADI,-,KATILMADI,--.--.----,--:--:--,Başbakan Ahmet Davutoğlu tarafından kurulan 64...,7'nci Birleşim
543,OSMANİYE,BAHÇELİ,Devlet,Milliyetçi Hareket Partisi,KATILMADI,RED,RED,--.--.----,--:--:--,Başbakan Ahmet Davutoğlu tarafından kurulan 64...,7'nci Birleşim


In [18]:
for year in all_legislative_year_urls:
    if year == "https://www.tbmm.gov.tr/tutanak/donem21/tutanak5.htm":
        pass
    else:
        print(year)
        get_year_page(year)

https://www.tbmm.gov.tr/tutanak/donem27/tutanak3.htm
https://www.tbmm.gov.tr/tutanak/donem27/yil3/ham/b23390oylama.htm
https://www.tbmm.gov.tr/tutanak/donem27/yil3/ham/b23389oylama.htm
https://www.tbmm.gov.tr/tutanak/donem27/yil3/ham/b23388oylama.htm
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
https://www.tbmm.gov.tr/tutanak/donem27/yil3/ham/b23374oylama.htm
it is empty
it is empty
it is empty
it is empty
it is empty
https://www.tbmm.gov.tr/tutanak/donem27/yil3/ham/b23366oylama.htm
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
it is empty
https://www.tbmm.gov.tr/tutanak/donem27/yil3/ham/b23352oylama.htm
it is empty
https://www.tbmm.gov.tr/tutanak/donem27/yil3/ham/b23350oylama.htm
it is empty
it is empty
it is empty
it is empty
it is empty
https://www.tbmm.gov.tr/tutanak/donem27/yil3/ham/b23342oylama.htm
it is empty
['https://www.tbmm.gov.

IndexError: list index out of range

In [111]:
all_legislative_year_urls[6:8]

['https://www.tbmm.gov.tr/tutanak/donem25/tutanak2.htm',
 'https://www.tbmm.gov.tr/tutanak/donem25/tutanak1.htm']